<a href="https://colab.research.google.com/github/mab2004/speech-to-text-api/blob/main/speech_to_text_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch fastapi "uvicorn[standard]"

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("YOUR_AUTH_TOKEN_HERE")

In [ ]:
import torch
import uvicorn
from fastapi import FastAPI, UploadFile, File
from transformers import pipeline
import nest_asyncio
from pyngrok import ngrok

# Use nest_asyncio to allow Uvicorn to run in a Colab environment
nest_asyncio.apply()

# --- API setup ---
app = FastAPI()

# --- Load the Whisper model ---
# Using a GPU if available
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device, return_timestamps=True)

# --- API endpoint for transcription ---
@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    # Read the uploaded audio file
    audio_data = await file.read()

    # Transcribe the audio using the Whisper model
    transcription = pipe(audio_data)["text"]

    return {"transcription": transcription}

# --- Start the API and create a public URL with ngrok ---
# This part runs the API and generates a public URL
ngrok_tunnel = ngrok.connect(8000)
print(f'Public URL: {ngrok_tunnel.public_url}')

uvicorn.run(app, port=8000)